In [112]:
import os

In [ ]:
ANTHROPIC_API_KEY = "Your API Key"

In [114]:
os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY

In [115]:
import json
import re
import csv
import time
from dataclasses import dataclass, field, asdict
from typing import List, Dict, Optional, Tuple, Any
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from tabulate import tabulate
from IPython.display import display, Markdown, HTML
import anthropic
from anthropic import Anthropic

In [116]:
class CoTLogger:
    """Logs all Chain of Thought processes for transparency and debugging"""

    def __init__(self, verbose: bool = True):
        self.verbose = verbose
        self.logs = []
        self.current_task = None

    def start_task(self, task_name: str):
        """Start logging a new task"""
        self.current_task = task_name
        timestamp = datetime.now().strftime("%H:%M:%S")
        entry = f"\n{'='*60}\n🎯 TASK: {task_name}\n⏰ TIME: {timestamp}\n{'='*60}"
        self.logs.append(entry)
        if self.verbose:
            print(entry)

    def log_step(self, step: str, details: str = ""):
        """Log a step in the current task"""
        entry = f"  ➤ {step}"
        if details:
            entry += f"\n    {details}"
        self.logs.append(entry)
        if self.verbose:
            print(entry)

    def log_prompt(self, prompt_type: str, prompt: str):
        """Log the prompt being sent"""
        entry = f"\n📝 PROMPT ({prompt_type}):\n{'-'*40}\n{prompt[:500]}{'...' if len(prompt) > 500 else ''}\n{'-'*40}"
        self.logs.append(entry)
        if self.verbose:
            print(entry)

    def log_response(self, response_type: str, response: Any):
        """Log the response received"""
        if isinstance(response, dict):
            response_str = json.dumps(response, indent=2)[:500]
        else:
            response_str = str(response)[:500]
        entry = f"\n✅ RESPONSE ({response_type}):\n{response_str}{'...' if len(str(response)) > 500 else ''}"
        self.logs.append(entry)
        if self.verbose:
            print(entry)

    def log_error(self, error: str):
        """Log an error"""
        entry = f"\n❌ ERROR: {error}"
        self.logs.append(entry)
        if self.verbose:
            print(entry)

    def get_full_log(self) -> str:
        """Get the complete log as a string"""
        return "\n".join(self.logs)

In [117]:
@dataclass
class VocabItem:
    """Single vocabulary item with all metadata"""
    word: str
    ipa: str
    pos: str
    definition: str
    synonyms: List[str]
    example: str
    mnemonic: str = ""
    next_review: str = ""
    interval_days: int = 1
    ease_factor: float = 2.5

@dataclass
class PracticeItem:
    """Practice exercises for vocabulary"""
    cloze_paragraph: str
    mcqs: List[Dict[str, Any]]
    quick_recall_prompts: List[str]

@dataclass
class StudyPack:
    """Complete study package for one session"""
    theme: str
    level: str
    timestamp: str
    vocab_items: List[VocabItem]
    story: str
    practice: PracticeItem
    srs_schedule: List[Dict[str, Any]]
    metadata: Dict[str, Any] = field(default_factory=dict)


In [118]:
class ClaudeClient:
    """Enhanced Claude API client with Chain of Thought logging"""

    def __init__(self, api_key: str, model: str = "claude-3-7-sonnet-latest",
                 temperature: float = 0.7, cot_logger: Optional[CoTLogger] = None):
        self.client = Anthropic(api_key=api_key)
        self.model = model
        self.temperature = temperature
        self.cot = cot_logger or CoTLogger()

    def text_task(self, system_prompt: str, user_prompt: str, max_retries: int = 3) -> str:
        """Execute a text generation task with CoT logging"""
        self.cot.log_step("Preparing text generation task")
        self.cot.log_prompt("System", system_prompt)
        self.cot.log_prompt("User", user_prompt)

        for attempt in range(max_retries):
            try:
                self.cot.log_step(f"API call attempt {attempt + 1}")
                response = self.client.messages.create(
                    model=self.model,
                    max_tokens=2000,
                    temperature=self.temperature,
                    system=system_prompt,
                    messages=[{"role": "user", "content": user_prompt}]
                )
                result = response.content[0].text
                self.cot.log_response("Text", result)
                return result
            except Exception as e:
                self.cot.log_error(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)

    def json_task(self, system_prompt: str, user_prompt: str, max_retries: int = 3) -> Dict:
        """Execute a JSON generation task with CoT logging and robust parsing"""
        self.cot.log_step("Preparing JSON generation task")

        # Enhance prompts for JSON
        json_system = system_prompt + "\nYou must respond with valid JSON only. No additional text."
        json_user = user_prompt + "\n\nRespond with valid JSON only."

        self.cot.log_prompt("System (JSON)", json_system)
        self.cot.log_prompt("User (JSON)", json_user)

        for attempt in range(max_retries):
            try:
                self.cot.log_step(f"API call attempt {attempt + 1}")
                response = self.client.messages.create(
                    model=self.model,
                    max_tokens=4000,
                    temperature=self.temperature,
                    system=json_system,
                    messages=[{"role": "user", "content": json_user}]
                )

                raw_text = response.content[0].text
                self.cot.log_step("Raw response received", f"Length: {len(raw_text)} chars")

                # Parse JSON with fallback methods
                result = self._parse_json_response(raw_text)
                self.cot.log_response("JSON", result)
                return result

            except Exception as e:
                self.cot.log_error(f"Attempt {attempt + 1} failed: {str(e)}")
                if attempt == max_retries - 1:
                    raise
                time.sleep(2 ** attempt)

    def _parse_json_response(self, text: str) -> Dict:
        """Robust JSON parsing with multiple fallback strategies"""
        self.cot.log_step("Parsing JSON response")

        # Strategy 1: Direct parse
        try:
            return json.loads(text)
        except:
            self.cot.log_step("Direct parse failed, trying cleanup")

        # Strategy 2: Remove code fences
        cleaned = re.sub(r'```json?\s*', '', text)
        cleaned = re.sub(r'```\s*$', '', cleaned)
        try:
            return json.loads(cleaned)
        except:
            self.cot.log_step("Code fence removal failed, extracting JSON object")

        # Strategy 3: Extract first JSON object
        match = re.search(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', text, re.DOTALL)
        if match:
            try:
                return json.loads(match.group())
            except:
                self.cot.log_step("JSON extraction failed")

        # Strategy 4: Return minimal valid JSON
        self.cot.log_error("All parsing strategies failed, returning empty dict")
        return {}


In [119]:
class PromptBuilder:
    """Builds prompts for different tasks with CoT logging"""

    def __init__(self, cot_logger: Optional[CoTLogger] = None):
        self.cot = cot_logger or CoTLogger()

    def vocab_json(self, theme: str, level: str) -> Tuple[str, str]:
        """Build vocabulary generation prompt"""
        self.cot.log_step(f"Building vocab prompt for theme='{theme}', level='{level}'")

        system = f"""You are an expert English vocabulary teacher specializing in {level} level content.
Your task is to generate exactly 10 vocabulary items related to the theme '{theme}'.
Each word must be appropriate for {level} learners and commonly used in real contexts.
Respond ONLY with valid JSON, no additional text or explanation."""

        user = f"""Generate exactly 10 vocabulary items for the theme '{theme}' at {level} level.

Return a JSON object with this exact structure:
{{
  "vocab_items": [
    {{
      "word": "example",
      "ipa": "/ɪɡˈzæmpəl/",
      "pos": "noun",
      "definition": "a thing characteristic of its kind or illustrating a general rule",
      "synonyms": ["instance", "case", "specimen"],
      "example": "This painting is a perfect example of Renaissance art."
    }}
  ]
}}

Requirements:
- Exactly 10 items in the array
- IPA pronunciation must be accurate
- POS must be: noun, verb, adjective, adverb, preposition, conjunction, or interjection
- Definition must be simple and clear for {level} learners
- Synonyms must be a list of 2-3 words
- Example sentence must clearly demonstrate the word's usage"""

        return system, user

    def mnemonics_json(self, words: List[str]) -> Tuple[str, str]:
        """Build mnemonics generation prompt"""
        self.cot.log_step(f"Building mnemonics prompt for {len(words)} words")

        system = """You are a memory expert specializing in creating visual, memorable mnemonics.
Create brief, vivid mental images that help learners remember word meanings.
Respond ONLY with valid JSON."""

        words_list = ", ".join([f'"{w}"' for w in words])
        user = f"""Create memorable mnemonics for these words: [{words_list}]

Return a JSON object:
{{
  "mnemonics": {{
    "word1": "vivid image-based memory hook",
    "word2": "another memorable association"
  }}
}}

Make each mnemonic:
- Visual and imaginative
- Connect to the word's meaning
- 10-20 words maximum
- Easy to visualize"""

        return system, user

    def story(self, words: List[str], tone: str, story_length: int) -> Tuple[str, str]:
        """Build story generation prompt"""
        self.cot.log_step(f"Building story prompt: tone='{tone}', length={story_length} words")

        system = f"""You are a creative writer who crafts engaging {tone} stories for language learners.
Write naturally flowing narratives that organically incorporate vocabulary words.
Make the story memorable and appropriate for adult learners."""

        words_formatted = ", ".join(words)
        user = f"""Write a {tone} story of approximately {story_length} words that naturally incorporates ALL of these vocabulary words: {words_formatted}

Requirements:
- Use ALL 10 words exactly once each
- Bold each vocabulary word when used: **word**
- Make the context clear enough to infer word meanings
- Keep the story engaging and cohesive
- Appropriate length: {story_length} words (±10%)
- Natural flow - don't force words awkwardly"""

        return system, user

    def practice_json(self, vocab_items: List[VocabItem], story: str) -> Tuple[str, str]:
        """Build practice exercises prompt"""
        self.cot.log_step("Building practice exercises prompt")

        system = """You are an expert in creating effective language learning exercises.
Design practice materials that reinforce vocabulary through varied, engaging activities.
Respond ONLY with valid JSON."""

        # Prepare vocab data
        words_data = [{"word": v.word, "definition": v.definition, "pos": v.pos}
                     for v in vocab_items]

        user = f"""Create practice exercises for these vocabulary items:
{json.dumps(words_data, indent=2)}

Using this story as context:
{story[:500]}...

Return a JSON object:
{{
  "cloze_paragraph": "A paragraph from the story with blanks (____) for all 10 vocabulary words",
  "mcqs": [
    {{
      "question": "What does 'word' mean in the context: '...'?",
      "options": ["a) correct", "b) distractor1", "c) distractor2", "d) distractor3"],
      "correct": "a",
      "explanation": "Brief explanation why this is correct"
    }}
  ],
  "quick_recall": [
    "Quick prompt or definition for rapid recall practice"
  ]
}}

Requirements:
- Cloze: Use story context, replace each vocab word with ____ exactly once
- MCQs: Exactly 10 questions, plausible distractors, clear explanations
- Quick recall: 10 brief prompts for rapid practice"""

        return system, user


In [120]:
class VocabAgent:
    """Main orchestrator for vocabulary learning with full CoT logging"""

    def __init__(self, api_key: str, model: str = "claude-3-7-sonnet-20250219",
                 temperature: float = 0.7, verbose_cot: bool = True):
        self.cot = CoTLogger(verbose=verbose_cot)
        self.claude = ClaudeClient(api_key, model, temperature, self.cot)
        self.prompts = PromptBuilder(self.cot)

    def generate_pack(self, theme: str = "daily life", level: str = "B1",
                     tone: str = "humorous", story_length: int = 200) -> StudyPack:
        """Generate complete study pack with full CoT logging"""

        self.cot.start_task("GENERATE COMPLETE STUDY PACK")
        self.cot.log_step("Configuration", f"theme={theme}, level={level}, tone={tone}, story_len={story_length}")

        try:
            # Step 1: Generate vocabulary
            self.cot.start_task("1. VOCABULARY GENERATION")
            system_v, user_v = self.prompts.vocab_json(theme, level)
            vocab_data = self.claude.json_task(system_v, user_v)

            vocab_items = []
            for item in vocab_data.get("vocab_items", []):
                # Normalize synonyms
                syns = item.get("synonyms", [])
                if isinstance(syns, str):
                    syns = [s.strip() for s in syns.split(",")]

                vocab_items.append(VocabItem(
                    word=item["word"],
                    ipa=item["ipa"],
                    pos=item["pos"],
                    definition=item["definition"],
                    synonyms=syns,
                    example=item["example"]
                ))

            self.cot.log_step(f"Generated {len(vocab_items)} vocabulary items")

            # Step 2: Generate mnemonics
            self.cot.start_task("2. MNEMONICS GENERATION")
            words = [v.word for v in vocab_items]
            system_m, user_m = self.prompts.mnemonics_json(words)
            mnemonics_data = self.claude.json_task(system_m, user_m)

            for vocab_item in vocab_items:
                vocab_item.mnemonic = mnemonics_data.get("mnemonics", {}).get(
                    vocab_item.word, "Visualize the word in action"
                )

            self.cot.log_step("Mnemonics added to all vocabulary items")

            # Step 3: Generate story
            self.cot.start_task("3. STORY GENERATION")
            system_s, user_s = self.prompts.story(words, tone, story_length)
            story = self.claude.text_task(system_s, user_s)

            self.cot.log_step(f"Generated story with {len(story.split())} words")

            # Step 4: Generate practice exercises
            self.cot.start_task("4. PRACTICE EXERCISES GENERATION")
            system_p, user_p = self.prompts.practice_json(vocab_items, story)
            practice_data = self.claude.json_task(system_p, user_p)

            practice = PracticeItem(
                cloze_paragraph=practice_data.get("cloze_paragraph", ""),
                mcqs=practice_data.get("mcqs", []),
                quick_recall_prompts=practice_data.get("quick_recall", [])
            )

            self.cot.log_step(f"Generated {len(practice.mcqs)} MCQs")

            # Step 5: Create SRS schedule
            self.cot.start_task("5. SRS SCHEDULE CREATION")
            srs_schedule = self._create_srs_schedule(vocab_items)
            self.cot.log_step("SRS schedule created with SM-2 lite algorithm")

            # Create study pack
            study_pack = StudyPack(
                theme=theme,
                level=level,
                timestamp=datetime.now().isoformat(),
                vocab_items=vocab_items,
                story=story,
                practice=practice,
                srs_schedule=srs_schedule,
                metadata={
                    "model": self.claude.model,
                    "temperature": self.claude.temperature,
                    "tone": tone,
                    "story_length": story_length
                }
            )

            self.cot.start_task("✨ STUDY PACK GENERATION COMPLETE")
            self.cot.log_step("Success", f"Generated pack with {len(vocab_items)} words")

            return study_pack

        except Exception as e:
            self.cot.log_error(f"Failed to generate study pack: {str(e)}")
            raise

    def _create_srs_schedule(self, vocab_items: List[VocabItem]) -> List[Dict[str, Any]]:
        """Create spaced repetition schedule"""
        intervals = [1, 3, 7, 16, 35]  # Days
        schedule = []

        for vocab_item in vocab_items:
            reviews = []
            current_date = datetime.now()

            for interval in intervals:
                current_date += timedelta(days=interval)
                reviews.append(current_date.strftime("%Y-%m-%d"))

            schedule.append({
                "word": vocab_item.word,
                "reviews": reviews,
                "current_interval": intervals[0],
                "tip": "Mark 'easy' to advance, 'hard' to repeat"
            })

        return schedule

    def export_markdown(self, pack: StudyPack, filepath: str = "study_pack.md"):
        """Export study pack as Markdown with CoT logging"""
        self.cot.start_task("EXPORT TO MARKDOWN")
        self.cot.log_step(f"Exporting to {filepath}")

        md_content = f"""# Vocabulary Study Pack: {pack.theme}
**Level:** {pack.level}
**Generated:** {pack.timestamp}

## 📚 Vocabulary Items

| Word | IPA | POS | Definition | Synonyms | Example |
|------|-----|-----|------------|----------|---------|
"""
        for v in pack.vocab_items:
            syns = ", ".join(v.synonyms)
            md_content += f"| **{v.word}** | {v.ipa} | {v.pos} | {v.definition} | {syns} | {v.example} |\n"

        md_content += f"\n## 🧠 Memory Anchors\n\n"
        for v in pack.vocab_items:
            md_content += f"- **{v.word}**: {v.mnemonic}\n"

        md_content += f"\n## 📖 Story\n\n{pack.story}\n"

        md_content += f"\n## 🎯 Practice\n\n### Cloze Exercise\n\n{pack.practice.cloze_paragraph}\n"

        md_content += "\n### Multiple Choice Questions\n\n"
        for i, mcq in enumerate(pack.practice.mcqs, 1):
            md_content += f"**Q{i}:** {mcq['question']}\n"
            for opt in mcq['options']:
                md_content += f"   {opt}\n"
            md_content += f"   **Answer:** {mcq['correct']}\n"
            md_content += f"   **Explanation:** {mcq['explanation']}\n\n"

        md_content += "\n## 📅 SRS Schedule\n\n"
        md_content += "| Word | Review Dates |\n|------|-------------|\n"
        for item in pack.srs_schedule:
            dates = " → ".join(item['reviews'])
            md_content += f"| {item['word']} | {dates} |\n"

        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(md_content)

        self.cot.log_step(f"Markdown exported successfully to {filepath}")
        return filepath

    def export_anki_csv(self, pack: StudyPack, filepath: str = "anki_cards.csv"):
        """Export as Anki-compatible CSV with CoT logging"""
        self.cot.start_task("EXPORT TO ANKI CSV")
        self.cot.log_step(f"Exporting to {filepath}")

        with open(filepath, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(["Front", "Back", "Tags", "Notes"])

            for v in pack.vocab_items:
                front = f"{v.word} ({v.pos})"
                back = f"{v.definition}<br><br>IPA: {v.ipa}<br>Synonyms: {', '.join(v.synonyms)}<br>Example: {v.example}<br><br>Memory hook: {v.mnemonic}"
                tags = f"vocab {pack.theme.replace(' ', '_')} {pack.level}"
                notes = v.example

                writer.writerow([front, back, tags, notes])

        self.cot.log_step(f"Anki CSV exported with {len(pack.vocab_items)} cards")
        return filepath

In [121]:
class NotebookDisplay:
    """Beautiful display utilities for Jupyter/Colab with CoT"""

    def __init__(self, cot_logger: Optional[CoTLogger] = None):
        self.cot = cot_logger or CoTLogger()

    def show_vocab_table(self, pack: StudyPack):
        """Display vocabulary as a formatted table"""
        self.cot.log_step("Displaying vocabulary table")

        data = []
        for v in pack.vocab_items:
            data.append([
                v.word,
                v.ipa,
                v.pos,
                v.definition,
                ", ".join(v.synonyms),
                v.example[:50] + "..."
            ])

        df = pd.DataFrame(data, columns=["Word", "IPA", "POS", "Definition", "Synonyms", "Example"])
        display(HTML(df.to_html(index=False, escape=False)))

    def show_story(self, story: str):
        """Display story with highlighted vocabulary"""
        self.cot.log_step("Displaying story")

        # Convert **word** to HTML bold
        html_story = re.sub(r'\*\*(\w+)\*\*', r'<b style="color: #e74c3c">\1</b>', story)
        display(Markdown(f"## 📖 Memory Anchor Story\n\n{html_story}"))

    def show_mnemonics(self, pack: StudyPack):
        """Display mnemonics in a collapsible format"""
        self.cot.log_step("Displaying mnemonics")

        html = "<h2>🧠 Memory Hooks</h2>"
        for v in pack.vocab_items:
            html += f"""
            <details style="margin: 10px 0; padding: 10px; border: 1px solid #ddd; border-radius: 5px;">
                <summary style="cursor: pointer; font-weight: bold; color: #2c3e50;">
                    {v.word} ({v.pos})
                </summary>
                <p style="margin-top: 10px; padding: 10px; background: #f8f9fa;">
                    {v.mnemonic}
                </p>
            </details>
            """
        display(HTML(html))

    def show_practice(self, pack: StudyPack):
        """Display practice exercises with collapsible answers"""
        self.cot.log_step("Displaying practice exercises")

        # Cloze
        display(Markdown(f"### Fill in the Blanks\n\n{pack.practice.cloze_paragraph}"))

        # MCQs with collapsible answers
        html = "<h3>📝 Multiple Choice Questions</h3>"
        for i, mcq in enumerate(pack.practice.mcqs, 1):
            options_html = "<br>".join(mcq['options'])
            html += f"""
            <div style="margin: 20px 0; padding: 15px; background: #f8f9fa; border-radius: 5px;">
                <p><b>Q{i}:</b> {mcq['question']}</p>
                <p>{options_html}</p>
                <details style="margin-top: 10px;">
                    <summary style="cursor: pointer; color: #27ae60; font-weight: bold;">
                        Show Answer
                    </summary>
                    <p style="margin-top: 10px;">
                        <b>Correct:</b> {mcq['correct']}<br>
                        <b>Explanation:</b> {mcq['explanation']}
                    </p>
                </details>
            </div>
            """
        display(HTML(html))

    def show_srs_schedule(self, pack: StudyPack):
        """Display SRS schedule as a table"""
        self.cot.log_step("Displaying SRS schedule")

        data = []
        for item in pack.srs_schedule:
            data.append([
                item['word'],
                " → ".join(item['reviews'][:3]) + "...",
                item['tip']
            ])

        df = pd.DataFrame(data, columns=["Word", "Review Schedule", "Tip"])
        display(HTML("<h2>📅 Spaced Repetition Schedule</h2>"))
        display(HTML(df.to_html(index=False, escape=False)))

    def show_full_pack(self, pack: StudyPack):
        """Display complete study pack"""
        self.cot.start_task("DISPLAY COMPLETE STUDY PACK")

        display(HTML(f"""
            <div style="padding: 20px; color: white; border-radius: 10px; margin-bottom: 20px;">
            <h1 style="color: white; margin: 0;">🎓 Vocabulary Study Pack</h1>
            <p style="margin: 10px 0;"><b>Theme:</b> {pack.theme} | <b>Level:</b> {pack.level}</p>
            <p style="margin: 0;"><b>Generated:</b> {pack.timestamp}</p>
        </div>
        """))

        self.show_vocab_table(pack)
        self.show_story(pack.story)
        self.show_mnemonics(pack)
        self.show_practice(pack)
        self.show_srs_schedule(pack)

        self.cot.log_step("Display complete")

In [122]:
class InteractiveQuiz:
    """Interactive MCQ quiz for notebook with CoT"""

    def __init__(self, pack: StudyPack, cot_logger: Optional[CoTLogger] = None):
        self.pack = pack
        self.cot = cot_logger or CoTLogger()
        self.score = 0
        self.total = 0

    def run_quiz(self, num_questions: int = 5):
        """Run interactive quiz in notebook"""
        self.cot.start_task("INTERACTIVE QUIZ")

        import random
        from IPython.display import clear_output
        import ipywidgets as widgets

        questions = random.sample(self.pack.practice.mcqs, min(num_questions, len(self.pack.practice.mcqs)))

        for i, mcq in enumerate(questions, 1):
            clear_output(wait=True)

            print(f"Question {i}/{num_questions}")
            print("=" * 50)
            print(mcq['question'])
            print()

            for opt in mcq['options']:
                print(opt)

            answer = input("\nYour answer (a/b/c/d): ").strip().lower()

            self.total += 1
            if answer == mcq['correct'].lower():
                self.score += 1
                print("✅ Correct!")
            else:
                print(f"❌ Wrong. The correct answer is {mcq['correct']}")

            print(f"Explanation: {mcq['explanation']}")
            input("\nPress Enter to continue...")

        clear_output(wait=True)
        self.cot.log_step(f"Quiz complete: {self.score}/{self.total}")

        print("🎯 Quiz Complete!")
        print("=" * 50)
        print(f"Your Score: {self.score}/{self.total} ({self.score/self.total*100:.1f}%)")

        if self.score == self.total:
            print("🏆 Perfect score! Excellent work!")
        elif self.score >= self.total * 0.8:
            print("🌟 Great job! You're mastering these words!")
        elif self.score >= self.total * 0.6:
            print("👍 Good effort! Keep practicing!")
        else:
            print("💪 Keep studying! You'll get there!")

In [123]:

def setup_colab():
    """Setup function for Google Colab environment"""
    print("🚀 Setting up English Learning AI Agent...")
    print("=" * 60)

    # Check for API key
    import os
    if 'ANTHROPIC_API_KEY' not in os.environ:
        print("⚠️  Please set your Anthropic API key:")
        print("   os.environ['ANTHROPIC_API_KEY'] = 'your-key-here'")
        return False

    print("✅ API key detected")
    print("✅ All dependencies loaded")
    print("✅ Ready to generate study packs!")
    print("=" * 60)
    return True

def quick_start_demo():
    """Quick start demonstration with Chain of Thought logging"""
    import os

    print("🎓 ENGLISH LEARNING AI AGENT - QUICK START DEMO")
    print("=" * 60)

    # Check setup
    if 'ANTHROPIC_API_KEY' not in os.environ:
        print("❌ Please set your API key first:")
        print("   os.environ['ANTHROPIC_API_KEY'] = 'your-key-here'")
        return

    # Create agent with CoT logging
    print("\n📊 Initializing Agent with Chain of Thought Logging...")
    agent = VocabAgent(
        api_key=os.environ['ANTHROPIC_API_KEY'],
        verbose_cot=True  # Enable verbose Chain of Thought logging
    )

    # Generate study pack
    print("\n🎯 Generating Study Pack...")
    print("Configuration:")
    print("  • Theme: Technology")
    print("  • Level: B2 (Upper-Intermediate)")
    print("  • Tone: Professional")
    print("  • Story Length: 200 words")
    print("\n" + "=" * 60)

    pack = agent.generate_pack(
        theme="technology",
        level="B2",
        tone="professional",
        story_length=200
    )

    # Display results
    print("\n📚 Displaying Study Pack...")
    display_helper = NotebookDisplay(agent.cot)
    display_helper.show_full_pack(pack)

    # Export files
    print("\n💾 Exporting Files...")
    md_file = agent.export_markdown(pack, "technology_study_pack.md")
    csv_file = agent.export_anki_csv(pack, "technology_anki_cards.csv")

    print(f"✅ Markdown exported: {md_file}")
    print(f"✅ Anki CSV exported: {csv_file}")

    # Show CoT summary
    print("\n" + "=" * 60)
    print("📋 CHAIN OF THOUGHT SUMMARY")
    print("=" * 60)
    print(f"Total steps logged: {len(agent.cot.logs)}")
    print("\nYou can access the full CoT log with: agent.cot.get_full_log()")

    return pack, agent

def custom_generation_with_cot(theme, level="B1", tone="conversational", story_length=200):
    """Generate a custom study pack with full CoT visibility"""
    import os

    if 'ANTHROPIC_API_KEY' not in os.environ:
        print("❌ Please set your API key first")
        return None

    print(f"🎯 Generating Custom Study Pack: {theme}")
    print("=" * 60)

    # Create agent with CoT
    agent = VocabAgent(
        api_key=os.environ['ANTHROPIC_API_KEY'],
        verbose_cot=True
    )

    # Generate pack
    pack = agent.generate_pack(theme, level, tone, story_length)

    # Display
    display_helper = NotebookDisplay(agent.cot)
    display_helper.show_full_pack(pack)

    # Export
    agent.export_markdown(pack, f"{theme.replace(' ', '_')}_pack.md")
    agent.export_anki_csv(pack, f"{theme.replace(' ', '_')}_anki.csv")

    print("\n✅ Generation complete!")
    print(f"📊 Chain of Thought steps: {len(agent.cot.logs)}")

    return pack, agent

In [124]:

PRESET_THEMES = {
    "beginner_friendly": [
        ("daily routines", "A1", "simple", 150),
        ("family and friends", "A2", "warm", 150),
        ("food and cooking", "A2", "conversational", 180),
        ("hobbies", "B1", "enthusiastic", 200)
    ],
    "business": [
        ("meetings", "B2", "professional", 200),
        ("negotiations", "C1", "formal", 250),
        ("presentations", "B2", "confident", 200),
        ("networking", "B2", "professional", 220)
    ],
    "academic": [
        ("research methods", "C1", "academic", 250),
        ("critical thinking", "B2", "analytical", 230),
        ("academic writing", "C1", "formal", 250),
        ("data analysis", "B2", "technical", 200)
    ],
    "creative": [
        ("art and design", "B1", "creative", 200),
        ("music and performance", "B2", "expressive", 220),
        ("storytelling", "B2", "imaginative", 250),
        ("photography", "B1", "descriptive", 200)
    ]
}

def generate_preset_pack(category="beginner_friendly", index=0):
    """Generate a study pack from preset configurations"""
    import os

    if category not in PRESET_THEMES:
        print(f"❌ Unknown category. Choose from: {list(PRESET_THEMES.keys())}")
        return None

    presets = PRESET_THEMES[category]
    if index >= len(presets):
        print(f"❌ Index out of range. Max index for {category}: {len(presets)-1}")
        return None

    theme, level, tone, length = presets[index]
    print(f"📚 Generating from preset: {category}[{index}]")

    return custom_generation_with_cot(theme, level, tone, length)

In [125]:
class ProgressTracker:
    """Track learning progress over time with CoT"""

    def __init__(self, cot_logger: Optional[CoTLogger] = None):
        self.cot = cot_logger or CoTLogger()
        self.history = []
        self.word_scores = {}

    def record_quiz_result(self, word: str, correct: bool):
        """Record a quiz result for a word"""
        if word not in self.word_scores:
            self.word_scores[word] = {"attempts": 0, "correct": 0}

        self.word_scores[word]["attempts"] += 1
        if correct:
            self.word_scores[word]["correct"] += 1

        self.cot.log_step(f"Recorded: {word} - {'✅' if correct else '❌'}")

    def get_word_accuracy(self, word: str) -> float:
        """Get accuracy percentage for a word"""
        if word not in self.word_scores:
            return 0.0

        score = self.word_scores[word]
        if score["attempts"] == 0:
            return 0.0

        return (score["correct"] / score["attempts"]) * 100

    def get_weak_words(self, threshold: float = 60.0) -> List[str]:
        """Get words below accuracy threshold"""
        weak = []
        for word, score in self.word_scores.items():
            if score["attempts"] > 0:
                accuracy = (score["correct"] / score["attempts"]) * 100
                if accuracy < threshold:
                    weak.append(word)

        self.cot.log_step(f"Found {len(weak)} weak words (< {threshold}% accuracy)")
        return weak

    def display_progress(self):
        """Display progress dashboard"""
        if not self.word_scores:
            print("No progress data yet. Take some quizzes first!")
            return

        print("📊 LEARNING PROGRESS DASHBOARD")
        print("=" * 60)

        # Overall stats
        total_attempts = sum(s["attempts"] for s in self.word_scores.values())
        total_correct = sum(s["correct"] for s in self.word_scores.values())
        overall_accuracy = (total_correct / total_attempts * 100) if total_attempts > 0 else 0

        print(f"📈 Overall Accuracy: {overall_accuracy:.1f}%")
        print(f"📝 Words Practiced: {len(self.word_scores)}")
        print(f"✍️ Total Attempts: {total_attempts}")
        print()

        # Word-level performance
        print("📚 Word Performance:")
        print("-" * 40)

        for word, score in sorted(self.word_scores.items(),
                                  key=lambda x: x[1]["correct"]/max(x[1]["attempts"], 1),
                                  reverse=True):
            accuracy = (score["correct"] / score["attempts"] * 100) if score["attempts"] > 0 else 0
            bar = "█" * int(accuracy / 10) + "░" * (10 - int(accuracy / 10))
            emoji = "🌟" if accuracy >= 80 else "📖" if accuracy >= 60 else "🔄"
            print(f"{emoji} {word:<15} {bar} {accuracy:.0f}% ({score['correct']}/{score['attempts']})")


In [126]:
pack, agent = custom_generation_with_cot("Sport", "C2", "professional", 200)

🎯 Generating Custom Study Pack: Sport

🎯 TASK: GENERATE COMPLETE STUDY PACK
⏰ TIME: 03:57:10
  ➤ Configuration
    theme=Sport, level=C2, tone=professional, story_len=200

🎯 TASK: 1. VOCABULARY GENERATION
⏰ TIME: 03:57:10
  ➤ Building vocab prompt for theme='Sport', level='C2'
  ➤ Preparing JSON generation task

📝 PROMPT (System (JSON)):
----------------------------------------
You are an expert English vocabulary teacher specializing in C2 level content.
Your task is to generate exactly 10 vocabulary items related to the theme 'Sport'.
Each word must be appropriate for C2 learners and commonly used in real contexts.
Respond ONLY with valid JSON, no additional text or explanation.
You must respond with valid JSON only. No additional text.
----------------------------------------

📝 PROMPT (User (JSON)):
----------------------------------------
Generate exactly 10 vocabulary items for the theme 'Sport' at C2 level.

Return a JSON object with this exact structure:
{
  "vocab_items": [
  

  ➤ Displaying vocabulary table


Word,IPA,POS,Definition,Synonyms,Example
trajectory,/trəˈdʒɛktəri/,noun,the path followed by a projectile or body moving under the action of given forces,"flight path, course, arc",The golfer carefully calculated the trajectory nee...
accolade,/ˈækəleɪd/,noun,an award or privilege granted as a special honor or as an acknowledgment of merit,"honor, tribute, laurel",Winning Olympic gold was the ultimate accolade in ...
tenacity,/təˈnæsəti/,noun,the quality of being determined to do something; persistence,"perseverance, determination, resolve",Her tenacity throughout the marathon inspired ever...
exemplify,/ɪɡˈzɛmplɪfaɪ/,verb,to be a typical example of something or to demonstrate a quality clearly,"embody, epitomize, personify",The captain's leadership during the championship f...
ameliorate,/əˈmiːliəreɪt/,verb,to make something better or more tolerable,"improve, enhance, upgrade",The new training regimen was designed to ameliorat...
indefatigable,/ˌɪndɪˈfætɪɡəbl/,adjective,persisting tirelessly; never giving up or getting tired,"tireless, unflagging, untiring",The indefatigable runner completed six marathons i...
proclivity,/prəˈklɪvəti/,noun,a natural inclination or tendency to behave in a particular way,"tendency, predisposition, penchant",His proclivity for taking risks made him an except...
paragon,/ˈpærəɡɒn/,noun,a person or thing regarded as a perfect example of a particular quality,"model, epitome, archetype",The coach described the retiring captain as a para...
sagacious,/səˈɡeɪʃəs/,adjective,having or showing keen mental discernment and good judgment,"astute, perceptive, shrewd",The sagacious chess player anticipated her opponen...
pugnacious,/pʌɡˈneɪʃəs/,adjective,"eager or quick to argue, quarrel, or fight","combative, aggressive, belligerent",The boxer's pugnacious style made him a formidable...


  ➤ Displaying story


## 📖 Memory Anchor Story

# The Rise of Mira Chen

Mira Chen's career <b style="color: #e74c3c">trajectory</b> began modestly in a small tech startup, where her natural <b style="color: #e74c3c">proclivity</b> for solving complex problems quickly distinguished her from peers. While her <b style="color: #e74c3c">pugnacious</b> competitors often clashed over recognition, Mira focused on results.

"Success isn't measured by a single <b style="color: #e74c3c">accolade</b>," her mentor advised with <b style="color: #e74c3c">sagacious</b> insight. "It's built through consistent excellence."

This philosophy helped Mira <b style="color: #e74c3c">exemplify</b> leadership when her team faced a critical software failure. Working tirelessly to <b style="color: #e74c3c">ameliorate</b> the situation, she demonstrated remarkable <b style="color: #e74c3c">tenacity</b>, refusing to accept defeat despite mounting pressure.

"Her approach was extraordinary," recalled former colleague James Whitfield. "She was <b style="color: #e74c3c">indefatigable</b>, working through three nights straight until finding the solution."

This crisis response catapulted Mira into senior leadership. Ten years later, as CTO of a Fortune 500 company, she's considered a <b style="color: #e74c3c">paragon</b> of technical leadership, having transformed the organization's entire development culture.

At a recent industry conference, Mira reflected on her journey: "Technical skills open doors, but resilience and adaptability determine how far you'll go. The challenges never stop—they just become more interesting."

...

  ➤ Displaying practice exercises


### Fill in the Blanks

Mira Chen's career ____ began modestly in a small tech startup, where her natural ____ for solving complex problems quickly distinguished her from peers. While her ____ competitors often clashed over recognition, Mira focused on results. "Success isn't measured by a single ____," her mentor advised with ____ insight. "It's built through consistent excellence." This philosophy helped Mira ____ leadership when her team faced challenges. Her ____ in pursuing solutions, even when others gave up, helped ____ difficult situations. She became an ____ worker, known for her unwavering determination. Eventually, Mira was recognized as a ____ of innovation in the industry.

  ➤ Displaying SRS schedule


Word,Review Schedule,Tip
trajectory,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
accolade,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
tenacity,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
exemplify,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
ameliorate,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
indefatigable,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
proclivity,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
paragon,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
sagacious,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"
pugnacious,2025-09-23 → 2025-09-26 → 2025-10-03...,"Mark 'easy' to advance, 'hard' to repeat"


  ➤ Display complete

🎯 TASK: EXPORT TO MARKDOWN
⏰ TIME: 03:58:04
  ➤ Exporting to Sport_pack.md
  ➤ Markdown exported successfully to Sport_pack.md

🎯 TASK: EXPORT TO ANKI CSV
⏰ TIME: 03:58:04
  ➤ Exporting to Sport_anki.csv
  ➤ Anki CSV exported with 10 cards

✅ Generation complete!
📊 Chain of Thought steps: 57
